In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## BERT-CNN-1D Model


In [ ]:
!pip install transformers==3.0.0
!pip install emoji

import gc
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from transformers import BertModel, BertTokenizer

class BERT_CNN_1D(nn.Module):
  def __init__(self):
    super(BERT_CNN_1D, self).__init__()
    self.bert = BertModel.from_pretrained('bert-base-uncased')
    self.conv = nn.Conv1d(in_channels=768, out_channels=64, kernel_size=3, padding="same")
    self.pool = nn.MaxPool1d(kernel_size=3, stride=1)
    self.relu = nn.ReLU()
    self.dropout = nn.Dropout(0.1)
    self.fc = nn.Linear(2176, 2) ## 64*34 = 2176
    self.flat = nn.Flatten()
    self.softmax = nn.LogSoftmax(dim=1)

  def forward(self, sent_id, mask): 
    last_hidden_state, _ = self.bert(sent_id, attention_mask=mask)
    x = last_hidden_state

    gc.collect()
    torch.cuda.empty_cache()

    x = self.dropout(x)
    x = torch.transpose(x, 1, 2)
    x = self.pool(self.dropout(self.relu(self.conv(x))))
    x = self.fc(self.dropout(self.flat(self.dropout(x))))
    return self.softmax(x)


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Importing the dataset and tokenization
The pre_process_dataset(values) and data_process(data,labels) methods have been implemented by [@ZeroxTM](https://github.com/ZeroxTM) in [this repository](https://github.com/ZeroxTM/BERT-CNN-Fine-Tuning-For-Hate-Speech-Detection-in-Online-Social-Media) the methods are in Pre_Process.py.

In [ ]:
import re
import string
import emoji

DATASET_PATH = '/content/drive/MyDrive/NLP-project/dataset/train.csv'

def balance_dataset(dataset):
    min_len = (dataset['label']==1).sum()
    sampled_from_0s = dataset[dataset['label']==0].sample(n = 2*min_len, random_state=42)
    return pd.concat([dataset[dataset['label']==1], sampled_from_0s])

def read_dataset(path):
    dataset = pd.read_csv(path)
    dataset = dataset[~pd.isna(dataset['comment_text'])]
    dataset['label'] = dataset.values[:,2:].sum(axis=1) > 0
    dataset['label'] = dataset['label'].astype(int)
    balanced_dataset = balance_dataset(dataset)
    return balanced_dataset['comment_text'].tolist(), balanced_dataset['label']

def data_process(data, labels):
    input_ids = []
    attention_masks = []
    bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
    for sentence in data:
        bert_inp = bert_tokenizer.__call__(sentence, max_length=36,
                                           padding='max_length', pad_to_max_length=True,
                                           truncation=True, return_token_type_ids=False)

        input_ids.append(bert_inp['input_ids'])
        attention_masks.append(bert_inp['attention_mask'])
    input_ids = np.asarray(input_ids)
    attention_masks = np.array(attention_masks)
    labels = np.array(labels)

    return input_ids, attention_masks, labels

def pre_process_dataset(values):
    new_values = list()
    # Emoticons
    emoticons = [':-)', ':)', '(:', '(-:', ':))', '((:', ':-D', ':D', 'X-D', 'XD', 'xD', 'xD', '<3', '</3', ':\*',
                 ';-)',
                 ';)', ';-D', ';D', '(;', '(-;', ':-(', ':(', '(:', '(-:', ':,(', ':\'(', ':"(', ':((', ':D', '=D',
                 '=)',
                 '(=', '=(', ')=', '=-O', 'O-=', ':o', 'o:', 'O:', 'O:', ':-o', 'o-:', ':P', ':p', ':S', ':s', ':@',
                 ':>',
                 ':<', '^_^', '^.^', '>.>', 'T_T', 'T-T', '-.-', '*.*', '~.~', ':*', ':-*', 'xP', 'XP', 'XP', 'Xp',
                 ':-|',
                 ':->', ':-<', '$_$', '8-)', ':-P', ':-p', '=P', '=p', ':*)', '*-*', 'B-)', 'O.o', 'X-(', ')-X']

    for value in values:
        # Remove dots
        text = value.replace(".", "").lower()
        text = re.sub(r"[^a-zA-Z?.!,¿]+", " ", text)
        users = re.findall("[@]\w+", text)
        for user in users:
            text = text.replace(user, "<user>")
        urls = re.findall(r'(https?://[^\s]+)', text)
        if len(urls) != 0:
            for url in urls:
                text = text.replace(url, "<url >")
        for emo in text:
            if emo in emoji.UNICODE_EMOJI:
                text = text.replace(emo, "<emoticon >")
        for emo in emoticons:
            text = text.replace(emo, "<emoticon >")
        numbers = re.findall('[0-9]+', text)
        for number in numbers:
            text = text.replace(number, "<number >")
        text = text.replace('#', "<hashtag >")
        text = re.sub(r"([?.!,¿])", r" ", text)
        text = "".join(l for l in text if l not in string.punctuation)
        text = re.sub(r'[" "]+', " ", text)
        new_values.append(text)
    return new_values

def load_and_process(path):
    data, labels = read_dataset(path)
    num_of_labels = len(labels.unique())
    input_ids, attention_masks, labels = data_process(pre_process_dataset(data), labels)

    return input_ids, attention_masks, labels


## Training phase

In [ ]:

def train():
    model.train()
    total_loss = 0
    total = len(train_dataloader)

    for i, batch in enumerate(train_dataloader):
        step = i+1
        percent = "{0:.3f}".format(100 * (step / float(total)))
        lossp = "{0:.3f}".format(total_loss/(step*batch_size))
        filledLength = int(100 * step // total)
        bar = '█' * filledLength + '>'  *(filledLength < 100) + '.' * (99 - filledLength)
        print(f'\rBatch {step}/{total} |{bar}| {percent}% complete, loss={lossp}', end='')
        batch = [r.to(device) for r in batch]
        sent_id, mask, labels = batch
        del batch
        gc.collect()
        torch.cuda.empty_cache()
        model.zero_grad()
        preds = model(sent_id.to(device).long(), mask)
        loss = cross_entropy(preds, labels)
        total_loss += float(loss.item())
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()

    gc.collect()
    torch.cuda.empty_cache()

    avg_loss = total_loss / (len(train_dataloader)*batch_size)
    
    return avg_loss


# function for evaluating the model
def evaluate():
    print("\n\nEvaluating...")
    model.eval()
    total_loss= 0
    total = len(val_dataloader)
    for i, batch in enumerate(val_dataloader):
        step = i+1
        percent = "{0:.3f}".format(100 * (step / float(total)))
        lossp = "{0:.3f}".format(total_loss/(step*batch_size))
        filledLength = int(100 * step // total)
        bar = '█' * filledLength + '>' * (filledLength < 100) + '.' * (99 - filledLength)
        print(f'\rBatch {step}/{total} |{bar}| {percent}% complete, loss={lossp}', end='')
        # push the batch to gpu
        batch = [t.to(device) for t in batch]
        sent_id, mask, labels = batch
        del batch
        gc.collect()
        torch.cuda.empty_cache()
        with torch.no_grad():
            preds = model(sent_id, mask)
            loss = cross_entropy(preds, labels)
            total_loss += float(loss.item())

    gc.collect()
    torch.cuda.empty_cache()
    val_avg_loss = total_loss / (len(val_dataloader)*batch_size)

    return val_avg_loss

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Load the dataset
input_ids, attention_masks, labels = load_and_process(DATASET_PATH)
df = pd.DataFrame(list(zip(input_ids, attention_masks)), columns=['input_ids', 'attention_masks'])

train_text, temp_text, train_labels, temp_labels = train_test_split(df, labels,
                             random_state=42, test_size=0.2, stratify=labels)

val_text, test_text, val_labels, test_labels = train_test_split(temp_text, temp_labels,
                         random_state=42, test_size=0.5, stratify=temp_labels)

del temp_text
gc.collect()
torch.cuda.empty_cache()

train_count = len(train_labels)
test_count = len(test_labels)
val_count = len(val_labels)


# for train set
train_seq = torch.tensor(train_text['input_ids'].tolist())
train_mask = torch.tensor(train_text['attention_masks'].tolist())
train_y = torch.tensor(train_labels.tolist())

# for validation set
val_seq = torch.tensor(val_text['input_ids'].tolist())
val_mask = torch.tensor(val_text['attention_masks'].tolist())
val_y = torch.tensor(val_labels.tolist())

# for test set
test_seq = torch.tensor(test_text['input_ids'].tolist())
test_mask = torch.tensor(test_text['attention_masks'].tolist())
test_y = torch.tensor(test_labels.tolist())

from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

batch_size = 32

train_data = TensorDataset(train_seq, train_mask, train_y)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

val_data = TensorDataset(val_seq, val_mask, val_y)
val_sampler = SequentialSampler(val_data)
val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=batch_size)

model = BERT_CNN_1D()

model = model.to(device)

# Adam optimizer
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=2e-5)

cross_entropy = nn.NLLLoss()

epochs = 3
current = 1
# for each epoch
while current <= epochs:

    print(f'\nEpoch {current} / {epochs}:')

    # train model
    train_loss = train()

    # evaluate model
    valid_loss = evaluate()
    
    print(f'\n\nTraining Loss: {train_loss:.3f}')
    print(f'Validation Loss: {valid_loss:.3f}')

    current = current + 1


Downloading:   0%|          | 0.00/433 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]


Epoch 1 / 3:
Batch 1217/1217 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.000% complete, loss=0.007

Evaluating...
Batch 153/153 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.000% complete, loss=0.006

Training Loss: 0.007
Validation Loss: 0.006

Epoch 2 / 3:
Batch 1217/1217 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.000% complete, loss=0.005

Evaluating...
Batch 153/153 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.000% complete, loss=0.007

Training Loss: 0.005
Validation Loss: 0.007

Epoch 3 / 3:
Batch 1217/1217 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.000% complete, loss=0.003

Evaluating...
Batch 153/153 |███████████████████████████████████████████████████████████████████

## Evaluation

In [ ]:
gc.collect()
torch.cuda.empty_cache()

with torch.no_grad():
    preds = model(test_seq.to(device), test_mask.to(device))
    preds = preds.detach().cpu().numpy()

preds = np.argmax(preds, axis=1)
print("Performance:")
print("Classification Report:")
print(classification_report(test_y, preds))

Performance:
Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.91      0.93      3245
           1       0.84      0.91      0.87      1623

    accuracy                           0.91      4868
   macro avg       0.90      0.91      0.90      4868
weighted avg       0.91      0.91      0.91      4868

